# Report results

In [1]:
import os
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

In [2]:
%matplotlib inline

### Datasets characteristics

In [3]:
datasets = glob.glob(os.path.join(os.getcwd(),f'data/*'))

In [4]:
datasets_data = pd.DataFrame()
for dataset in datasets:
    name = dataset.rsplit('/', 1)[-1]
    df_path_train = os.path.join(dataset, 'split', 'train.csv')
    df_path_test = os.path.join(dataset, 'split', 'test.csv')
    df_train = pd.read_csv(df_path_train)
    df_test = pd.read_csv(df_path_test)
    df = pd.concat([df_train, df_test])
    df = df.reset_index(drop=True)
    df['basket'] = df['basket'].apply(lambda x: x.strip('][').split(', '))
    df['basket'] = df['basket'].apply(lambda x: [int(item) for item in x])
    df['basket_size'] = df.apply(lambda row: len(row['basket']), axis=1)
    avg_basket_size = df['basket_size'].mean()
    num_baskets_per_user = df.groupby('user_id')['basket'].count().mean()
    min_basket_size = df['basket_size'].min()
    max_basket_size = df['basket_size'].max()
    num_users = df['user_id'].nunique()
    num_baskets = len(df)

    exploded = df.explode('basket').rename({'basket': 'item_id'}, axis=1)
    num_items = exploded['item_id'].nunique()

    datasets_data = datasets_data._append({'Dataset': name, '#Users': num_users, '#Items': num_items, '#Baskets': num_baskets, 'Avg. basket size': avg_basket_size, '#Baskets per user': num_baskets_per_user, 'Min. basket size': min_basket_size, 'Max. basket size': max_basket_size}, ignore_index=True)

In [5]:
datasets_data.to_csv(os.path.join(os.getcwd(),f'report_results/datasets_data.csv'), index=False)

### Optimal hyperparameters

In [6]:
data = [
    ['instacart', 900, 0.9, 0.7, 3, 0.9],
    ['tafeng', 300, 0.9, 0.7, 7, 0.7],
    ['dunnhumby', 900, 0.9, 0.6, 3, 0.2],
    ['valuedshopper', 300, 1, 0.6, 7, 0.7],
    ['tmall', 100, 0.6, 0.8, 18, 0.7],
    ['taobao', 300, 0.6, 0.8, 10, 0.1]
]

columns = ['Dataset', 'Num Nearest Neighbors', 'Within decay rate', 'Group decay rate', 'Group count', 'Alpha']

hyperparams = pd.DataFrame(data, columns=columns)
hyperparams.to_csv(os.path.join(os.getcwd(),f'report_results/optimal_hyperparameters.csv'), index=False)


### Fairness

In [8]:
# Specify the model for which you want to create results
model = 'tifuknn' # 'top_personal', 'betavae'

#### Metrics vs. average basket size

In [7]:
datasets = [dataset.split('/')[-1] for dataset in datasets]

In [9]:
metrics = ['recall@010', 'recall@020', 'ndcg@010', 'ndcg@020', 'accuracy@010', 'accuracy@020',
 'precision@010', 'precision@020', 'f1@010', 'f1@020', 'mrr@010', 'mrr@020', 'phr@010', 'phr@020']
for metric in metrics:
    fig, axs = plt.subplots(2, 3, figsize=(12, 8))

    for dataset, ax in zip(datasets, axs.flat):
        path = glob.glob(os.path.join(os.getcwd(), f'results/{model}_{dataset}_*_user_metrics.csv'))
        df = pd.read_csv(path[0])
        train_df = pd.read_csv(os.path.join(os.getcwd(), f'data/{dataset}/split/train.csv'))
        train_df['basket_size'] = train_df.apply(lambda row: len(row['basket'].split(',')), axis=1)
        basket_size = train_df.groupby('user_id')['basket_size'].mean().reset_index()
        df = df.merge(basket_size, on=['user_id'])
        if dataset == 'dunnhumby':
            df['bin'] = pd.cut(df['basket_size'], list(range(0, 60, 5)))
        else:
            df['bin'] = pd.cut(df['basket_size'], list(range(0, min(5 * round(max(df['basket_size']) / 5), 100), 5)))

        fairness = df.groupby('bin').agg({metric: 'mean', 'user_id': 'count'})
        bar = fairness[metric].plot(kind='bar', color='y', label=f'avg. {metric}', ax=ax)
        ax2 = ax.twinx()
        line = fairness['user_id'].plot(kind='line', marker='d', secondary_y=True, label='#users', ax=ax2)
        ax.set_title(dataset)  # Set subplot title

        # Create proxy artists for the legend
        proxy_bar = mpatches.Patch(color='y', label=f'avg. {metric}')
        proxy_line = mlines.Line2D([], [], marker='d', label='#users')
        lines = [proxy_bar, proxy_line]
        labels = [line.get_label() for line in lines]
        ax.legend(lines, labels, loc='upper right')

    plt.tight_layout()
    plt.savefig(os.path.join(os.getcwd(), f'report_results/fairness_basket_size/{model}_{metric}.png'))
    plt.close()

#### Metrics vs. % of popular items

In [10]:
metrics = ['recall@010', 'recall@020', 'ndcg@010', 'ndcg@020', 'accuracy@010', 'accuracy@020',
           'precision@010', 'precision@020', 'f1@010', 'f1@020', 'mrr@010', 'mrr@020', 'phr@010', 'phr@020']

for metric in metrics:
    fig, axs = plt.subplots(2, 3, figsize=(12, 8))

    for i, dataset in enumerate(datasets):
        # Find ids of top x items
        interactions = pd.read_csv(os.path.join(os.getcwd(), f'data/{dataset}/processed/interactions.csv'))
        top_5 = int(0.20 * len(interactions['item_id']))
        item_count = interactions['item_id'].value_counts().reset_index()
        item_count['sum_cum'] = item_count['count'].cumsum()
        top_5_ids = list(item_count[item_count['sum_cum'] <= top_5].item_id)

        # Calculate percentage of popular items per user
        train_df = pd.read_csv(os.path.join(os.getcwd(), f'data/{dataset}/split/train.csv'))
        train_df['basket'] = train_df['basket'].apply(lambda x: x.strip('][').split(', '))
        train_df['basket'] = train_df['basket'].apply(lambda x: [int(item) for item in x])
        train_df['percent_of_popular_items'] = train_df['basket'].apply(
            lambda x: len([item for item in x if item in top_5_ids]) / len(x))
        popular_items = train_df.groupby('user_id')['percent_of_popular_items'].mean().reset_index()

        # Merge with metrics
        path = glob.glob(os.path.join(os.getcwd(), f'results/{model}_{dataset}_*_user_metrics.csv'))
        df = pd.read_csv(path[0])
        df = df.merge(popular_items, on=['user_id'])

        # Plot the results
        df['bin'] = pd.cut(df['percent_of_popular_items'], np.arange(0, 1.1, 0.1), include_lowest=True)
        df['bin'] = df['bin'].astype(str)
        df['bin'] = df['bin'].replace('(-0.001, 0.1]', '(0, 0.1]')

        ax = axs[i // 3, i % 3]
        fairness = df.groupby('bin').agg({metric: 'mean', 'user_id': 'count'})
        fairness[metric].plot(kind='bar', color='y', label=f'avg. {metric}', ax=ax)
        ax2 = ax.twinx()
        fairness['user_id'].plot(kind='line', marker='d', secondary_y=True, label='#users', ax=ax2)
        ax.set_xlabel('Percentage of Popular Items')
        ax.set_ylabel(f'Average {metric}')
        ax2.set_ylabel('#users')
        ax.set_title(dataset)

        # Create proxy artists for the legend
        proxy_bar = mpatches.Patch(color='y', label=f'avg. {metric}')
        proxy_line = mlines.Line2D([], [], marker='d', label='#users')
        lines = [proxy_bar, proxy_line]
        labels = [line.get_label() for line in lines]
        ax.legend(lines, labels, loc='upper right')

    plt.tight_layout()
    plt.savefig(os.path.join(os.getcwd(), f'report_results/fairness_popular_items/{model}_{metric}.png'))
    plt.close()

#### Metrics vs. novelty

In [11]:
# Calculate percentage of popular items per user
res = {}
for dataset in datasets:
    train_df = pd.read_csv(os.path.join(os.getcwd(), f'data/{dataset}/split/train.csv'))
    train_df['basket'] = train_df['basket'].apply(lambda x: x.strip('][').split(', '))
    train_df['basket'] = train_df['basket'].apply(lambda x: [int(item) for item in x])

    test_df = pd.read_csv(os.path.join(os.getcwd(), f'data/{dataset}/split/test.csv'))
    test_df['basket'] = test_df['basket'].apply(lambda x: x.strip('][').split(', '))
    test_df['basket'] = test_df['basket'].apply(lambda x: [int(item) for item in x])

    past_items = train_df.groupby('user_id')['basket'].sum().reset_index()
    past_items['past_items'] = past_items['basket'].apply(lambda x: list(set(x)))
    past_items = past_items.drop(columns = ['basket'])
    test_df = test_df.merge(past_items, on=['user_id'])
    test_df['novelty'] = test_df.apply(lambda row: len([item for item in row['basket'] if item not in row['past_items']]) / len(row['basket']), axis=1)
    novelty = test_df[['user_id', 'novelty']]

    path = glob.glob(os.path.join(os.getcwd(), f'results/{model}_{dataset}_*_user_metrics.csv'))
    df = pd.read_csv(path[0])
    df = df.merge(novelty, on=['user_id'])

    df['bin'] = pd.cut(df['novelty'], np.arange(0, 1.1, 0.1), include_lowest=True)
    df['bin'] = df['bin'].astype(str)
    df['bin'] = df['bin'].replace('(-0.001, 0.1]', '(0, 0.1]')

    res[dataset] = df

for metric in metrics:
    fig, axs = plt.subplots(2, 3, figsize=(12, 8))
    for i, (dataset, df) in enumerate(res.items()):
        ax = axs[i // 3, i % 3]
        fairness = df.groupby('bin').agg({metric: 'mean', 'user_id': 'count'})
        fairness[metric].plot(kind='bar', color='y', label=f'avg. {metric}', ax=ax)
        ax2 = ax.twinx()
        fairness['user_id'].plot(kind='line', marker='d', secondary_y=True, label='#users', ax=ax2)
        ax.set_xlabel('Percentage of unseen items')
        ax.set_ylabel(f'Average {metric}')
        ax2.set_ylabel('#users')
        ax.set_title(dataset)

        # Create proxy artists for the legend
        proxy_bar = mpatches.Patch(color='y', label=f'avg. {metric}')
        proxy_line = mlines.Line2D([], [], marker='d', label='#users')
        lines = [proxy_bar, proxy_line]
        labels = [line.get_label() for line in lines]
        ax.legend(lines, labels, loc='upper right')

    plt.tight_layout()
    plt.savefig(os.path.join(os.getcwd(), f'report_results/fairness_novelty/{model}_{metric}.png'))
    plt.close()
